In [1]:
!pip install astropy

import numpy as np
import pandas as pd
import astropy as ap
import matplotlib.pyplot as plt

In [2]:
# mount drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# loading in data

csv_data = '/content/drive/MyDrive/2024project/DES-SN5YR_HD.csv'
df = pd.read_csv(csv_data)

z_obs = df['zHD']
mu_obs = df['MU']
mu_error = df['MUERR_FINAL']

In [4]:
# mask to cut out high z
mask = z_obs < 0.85 # same as DES

z_obs_cut = z_obs[mask]
mu_obs_cut = mu_obs[mask]
mu_error_cut = mu_error[mask]

print(len(z_obs_cut))
print(len(mu_obs_cut))
print(len(mu_error_cut))

1747
1747
1747


In [5]:
# apply cov mat
sys_data = np.loadtxt('/content/drive/MyDrive/2024project/STAT+SYS_new.txt')
shape = (1829, 1829)
sys_data_proper = sys_data.reshape(shape)
sys_data_cut = sys_data_proper[mask][:,mask]
np.fill_diagonal(sys_data_cut, sys_data_cut.diagonal()+mu_error_cut**2)

inverse_cm = np.linalg.inv(sys_data_cut)

In [6]:
from scipy.stats import linregress
from astropy.io import fits
from scipy.integrate import quad
from scipy.optimize import curve_fit, minimize, approx_fprime
import csv

In [7]:
# minimising chi2 for Flat-ΛCDM
c = 3e5 # km/s

# flat-ΛCDM, fitting for omega_M using eqn from table 1
def E(z, OmegaM):
    hubble_param = np.sqrt(OmegaM * (1 + z)**3 + (1 - OmegaM))
    return hubble_param
    print(hubble_param)

# R0 * χ(z): Comoving distance, equation 2
def R0_chi(z, OmegaM, H0):
    def integrand(z):
        return 1 / E(z, OmegaM)

    # performing numerical integration
    integral, _ = quad(integrand, 0, z)
    return (c / H0) * integral

def mu_theoretical(OmegaM, H0, z):
  dL = (1+z) * (R0_chi(z, OmegaM, H0)) # changed z_obs to z
  mu = 5* np.log10(dL) + 25 # theoretical mu
  return mu

def chi2(params):
  OmegaM, H0 = params
  mu_model = np.array([mu_theoretical(OmegaM, H0, z) for z in z_obs_cut])

  dmu = mu_obs_cut - mu_model # observed mu - theoretical mu
  # transpose of dmu
  dmu_T = dmu.transpose()
  return np.dot(dmu, np.dot(inverse_cm, dmu_T))

initial_guess = [0.3, 50.0]  # OmegaM, H0 (H0 in km/s/Mpc)
result = minimize(chi2, x0 = initial_guess, method='nelder-mead')

print("Optimisation Results:")
print(f"OmegaM = {result.x[0]:.4f}")
print(f"H0 = {result.x[1]:.4f} km/s/Mpc")
print(f"Minimum chi-squared = {result.fun:.2f}")

Optimisation Results:
OmegaM = 0.3409
H0 = 69.4833 km/s/Mpc
Minimum chi-squared = 1542.16


In [8]:
# Flat-wCDM model

#cmbR prior values (this is what DES did too)
R_obs = 1.7502
sigma_R = 0.0046
z_star = 1089.95 # redshift of last scattering

def E(z, OmegaM, w):
    return np.sqrt(OmegaM * (1 + z)**3 + (1 - OmegaM) * (1 + z)**(3 * (1 + w)))

# R0 * χ(z): Comoving distance
def R0_chi(z, OmegaM, w, H0):
    def integrand(z):
        return 1 / E(z, OmegaM, w)

    integral, _ = quad(integrand, 0, z)
    return (c / H0) * integral

# defining R_model to add CMB-R as a prior
def R_model(OmegaM, w, H0):
    D_M = R0_chi(z_star, OmegaM, w, H0)
    return np.sqrt(OmegaM) * H0 * D_M / c

# Theoretical distance modulus
def mu_theoretical(OmegaM, w, H0, z):
    dL = (1 + z) * R0_chi(z, OmegaM, w, H0)
    return 5 * np.log10(dL) + 25

# Chi-squared function
def chi2(params):
    OmegaM, w, H0 = params
    mu_model = np.array([mu_theoretical(OmegaM, w, H0, z) for z in z_obs_cut])

    dmu = mu_obs_cut - mu_model  # observed - theoretical
    dmu_T = dmu.transpose()
    chi2_SN = np.dot(dmu, np.dot(inverse_cm, dmu_T))

    # CMB-R chi2
    R_true = R_model(OmegaM, w, H0)
    chi2_CMB = ((R_true - R_obs) / sigma_R)**2
    return chi2_SN + chi2_CMB

initial_guess = [0.3, -1.0, 50.0]  # OmegaM, w, H0
result2 = minimize(chi2, x0=initial_guess, method='nelder-mead')

print("Optimisation Results:")
print(f"OmegaM = {result2.x[0]:.4f}")
print(f"w = {result2.x[1]:.4f}")
print(f"H0 = {result2.x[2]:.4f} km/s/Mpc")
print(f"Minimum chi-squared = {result2.fun:.2f}")

Optimisation Results:
OmegaM = 0.3193
w = -0.9317
H0 = 69.3072 km/s/Mpc
Minimum chi-squared = 1540.62


In [9]:
# Flat-w0waCDM model

def E(z, OmegaM, w0, wa):
    exponent = -3 * wa * z / (1 + z)
    return np.sqrt(OmegaM * (1 + z)**3 + (1 - OmegaM) * (1 + z)**(3 * (1 + w0 + wa)) * np.exp(exponent))

# R0 * χ(z): Comoving distance
def R0_chi(z, OmegaM, w0, wa, H0):
    def integrand(z):
        return 1 / E(z, OmegaM, w0, wa)

    integral, _ = quad(integrand, 0, z)
    return (c / H0) * integral

# Theoretical distance modulus
def mu_theoretical(OmegaM, w0, wa, H0, z):
    dL = (1 + z) * R0_chi(z, OmegaM, w0, wa, H0)
    return 5 * np.log10(dL) + 25

# Chi-squared function
def chi2(params):
    OmegaM, w0, wa, H0 = params
    mu_model = np.array([mu_theoretical(OmegaM, w0, wa, H0, z) for z in z_obs_cut])

    dmu = mu_obs_cut - mu_model  # observed - theoretical
    dmu_T = dmu.transpose()
    return np.dot(dmu, np.dot(inverse_cm, dmu_T))

initial_guess = [0.3, -0.3, -8, 50.0]  # OmegaM, w0, wa, H0
result3 = minimize(chi2, x0=initial_guess, method='nelder-mead')

print("Optimisation Results:")
print(f"OmegaM = {result3.x[0]:.4f}")
print(f"w0 = {result3.x[1]:.4f}")
print(f"wa = {result3.x[2]:.4f}")
print(f"H0 = {result3.x[3]:.4f} km/s/Mpc")
print(f"Minimum chi-squared = {result3.fun:.2f}")

Optimisation Results:
OmegaM = 0.3613
w0 = -0.6481
wa = -2.5049
H0 = 68.5775 km/s/Mpc
Minimum chi-squared = 1535.06


In [10]:
import scipy
from scipy.stats import norm

delta_chi2 = result.fun - result3.fun # always do simpler model - complexer model
dof = 1 # difference in number of params between models

p_value = 1 - scipy.stats.chi2.cdf(delta_chi2, dof)

sigma = norm.ppf(1 - p_value/2)
print("Two-tailed p-value:", p_value)
print("Significance level (σ):", sigma)

Two-tailed p-value: 0.007705416196564174
Significance level (σ): 2.664712520675524


In [11]:
delta_chi2 = result.fun - result2.fun # always do simpler model - complexer model
dof = 1 # difference in number of params between models

p_value = 1 - scipy.stats.chi2.cdf(delta_chi2, dof)

sigma = norm.ppf(1 - p_value/2)
print("Two-tailed p-value:", p_value)
print("Significance level (σ):", sigma)

Two-tailed p-value: 0.2136702009150092
Significance level (σ): 1.243536495846231


In [12]:
delta_chi2 = result2.fun - result3.fun # always do simpler model - complexer model
dof = 2 # difference in number of params between models

p_value = 1 - scipy.stats.chi2.cdf(delta_chi2, dof)

sigma = norm.ppf(1 - p_value/2)
print("Two-tailed p-value:", p_value)
print("Significance level (σ):", sigma)

Two-tailed p-value: 0.062215264419466565
Significance level (σ): 1.8647584879796515
